In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
from sklearn.metrics.pairwise import cosine_similarity
import math
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)

sql_select = '''select created_at, location, tweet_content, cleaned_tweet, hashtags from tweets where hashtags != 'None';'''

cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)
df.reset_index(inplace = True)

In [3]:
df['created_datetime'] = pd.to_datetime(df['created_at'])
df['year'] = df.created_datetime.apply(lambda x: x.year)
df['month'] = df.created_datetime.apply(lambda x: x.month)
df['day'] = df.created_datetime.apply(lambda x: x.day)
#df['dayofweek'] = df.created_datetime.apply(lambda x: x.dayofweek)
df['hour'] = df.created_datetime.apply(lambda x: x.hour)

In [4]:
import pickle
!pip install redis
import redis
redis_ip = '34.211.59.66'
r = redis.StrictRedis(redis_ip)
r.keys()

    100% |████████████████████████████████| 61kB 2.2MB/s ta 0:00:011
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


[b'tweet_tfidf_fit_transform',
 b'hashtags_countvec_fit_transform',
 b'tweet_SVD_fit',
 b'tweet_tfidf_fit',
 b'hashtags_tfidf_fit_transform',
 b'hashtags_countvec_fit']

In [ ]:
df['hashtags'].str.contains('nbafinals').mean()

In [ ]:
hastages_series = df['hashtags'][df['hashtags'].isnull() == False]
len(hastages_series)

In [5]:
hashtags_countvec = pickle.loads(r.get('hashtags_countvec_fit'))

In [ ]:
hashtag_name = hashtags_countvec.get_feature_names()
hashtag_name[5000]

In [6]:
time_delta = max(df['created_datetime']) - min(df['created_datetime'])
time_window = time_delta.components.days*24 + time_delta.components.hours
time_lag = timedelta(hours = 12)
time_gap = timedelta(hours = 6)
windows = int(round(time_delta/time_gap,0))
print('start time: ', min(df['created_datetime']),'\n',\
      'end time:  ', max(df['created_datetime']),'\n',\
      'total hours: ',time_window,'\n',\
      'time lag: ', time_lag,'\n',\
      'time gap: ', time_gap,'\n',\
      'time windows: ', windows)

start time:  2017-05-31 23:50:46 
 end time:   2017-06-07 14:24:48 
 total hours:  158 
 time lag:  12:00:00 
 time gap:  6:00:00 
 time windows:  26


In [ ]:
start_time = min(df['created_datetime'])
start_time = start_time
end_time = start_time + time_lag
subset = df['hashtags'][((df['created_datetime'] < end_time) & (df['created_datetime'] > start_time))]
arr = hashtags_countvec.transform(subset).toarray()
hashtag_vec = pd.DataFrame(arr, \
                           index=subset,\
                           columns=hashtags_countvec.get_feature_names())

In [7]:
start_time = min(df['created_datetime'])
arr_list = []
for window in tqdm(range(windows)):
    start_time = start_time
    end_time = start_time + time_lag
    subset = df['hashtags'][((df['created_datetime'] < end_time) & (df['created_datetime'] > start_time))]
    arr = hashtags_countvec.transform(subset).toarray()
    arr_mean = arr.mean(axis = 0, dtype=np.float32)
    arr_list.append(arr_mean)
    start_time += time_gap

 12%|█▏        | 3/26 [00:02<00:19,  1.19it/s]


MemoryError: 

In [ ]:
arr_list.shape

In [ ]:
mean_hashtag = np.array(arr_list)
mean_hashtag.shape

In [ ]:
hashtag_mean_arr = pickle.dumps(mean_hashtag)
r.set('hashtag_mean_arr', hashtag_mean_arr)

In [ ]:
np.mean(mean_hashtag[37]*1000)

In [ ]:
np.max(mean_hashtag[mean_hashtag != 0.083333336], axis=0).max()

In [ ]:
hash_grad = np.gradient(mean_hashtag)[1]

In [ ]:
np.max(hash_grad, axis=0)

In [ ]:
transform = mean_hashtag.T
transform.shape

In [ ]:
transform[0].shape

In [ ]:
for i in range(100):
    plt.plot(mean_hashtag.T[i]*1000000)
plt.ylim(-.001,.001)
plt.show()

In [ ]:
mean_hashtag.shape

In [ ]:
type(mean_hashtag[1][53000])

In [ ]:
h_list = []
#for j in range(10):
for j in range(mean_hashtag.shape[1]):
    #a[j] = []
    h = [mean_hashtag[i][j] for i,j in zip(range(mean_hashtag.shape[0]),range(mean_hashtag.shape[1]))]
    h_list.append(h)

In [ ]:
type(h)

In [ ]:
mean_hashtag_1 = np.array(h_list)

In [ ]:
max(mean_hashtag_1[5000])